In [1]:
# Dependencies
import pandas as pd
import numpy as np
import os
import psycopg2

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, inspect, func
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy.orm import Session
from sqlalchemy import Column, Integer, String, Float

In [2]:
# Import Matplot lib
import matplotlib
from matplotlib import style
style.use('fivethirtyeight')
%matplotlib inline
# from matplotlib import style
# style.use('seaborn')
import matplotlib.pyplot as plt
import datetime as dt

In [3]:
from scipy import stats

## Load US Military Bases CSVs

In [6]:
## Load US Military Bases CSV (Alabama-Missouri)
#military_bases_1_file = "data/csv/military_bases/military_bases_1.csv"
#military_bases_df = pd.read_csv(military_bases_1_file)
#military_bases_df.head()

,OBJECTID,COMPONENT,SITE_NAME,JOINT_BASE,STATE_TERR,COUNTRY,OPER_STAT,PERIMETER,AREA,Shape_Length,Shape_Area
0,4,Army Active,Redstone Arsenal,NaN,Alabama,United States,Active,36.824373,59.656365,73710.271660,2.287661e+08
1,11,Army Active,Anniston Army Depot,NaN,Alabama,United States,Active,23.965883,24.341345,47656.478150,9.122207e+07
2,65,Army Active,Allen Stagefield AL,NaN,Alabama,United States,Active,1.641383,0.176575,3170.633316,6.274240e+05
3,66,Army Active,Brown 4b Stagefield AL,NaN,Alabama,United States,Active,1.984040,0.255045,3834.680625,9.092164e+05
4,67,Army Active,Cairns Basefield AL,NaN,Alabama,United States,Active,7.260375,2.015903,14027.722290,7.170286e+06


In [7]:
## Load US Military Bases CSV (Montana-Wyoming)
#military_bases_2_file = "data/csv/military_bases/military_bases_2.csv"
#military_bases_df = pd.read_csv(military_bases_2_file)
#military_bases_df.head()

,OBJECTID,COMPONENT,SITE_NAME,JOINT_BASE,STATE_TERR,COUNTRY,OPER_STAT,PERIMETER,AREA,Shape_Length,Shape_Area
0,2060,AF Active,Malmstrom AFB,NaN,Montana,United States,Active,11.742872,5.119799,28870.497100,2.904230e+07
1,2811,AF Guard,Great Falls IAP,NaN,Montana,United States,Active,2.266567,0.208855,5536.050297,1.183414e+06
2,3927,Army Guard,NG MTA Ft Wm Henry Harrison,NaN,Montana,United States,Active,21.550542,10.251946,52091.067920,5.627799e+07
3,3930,Army Guard,NG Helena Aviation RC- AASF- C12,NaN,Montana,United States,Active,1.259258,0.093633,3045.747510,5.136674e+05
4,4336,Army Guard,NG MTA Limestone Hills,NaN,Montana,United States,Active,29.450764,33.308389,70422.019250,1.805126e+08


## Load National UFO Reporting Center CSV (April 2020)

In [8]:
## Load National Ufo Reporting Center Data for April 2020
#nurc_april_file = "data/csv/nat_ufo_rep_ctr_0420.csv"
#nurc_april_df = pd.read_csv(nurc_april_file)
#nurc_april_df.head()

,datetime,city,state,shape,duration,summary,posted
0,4/30/2020 0:00,Thibodaux,LA,Light,5,"14 Objects in a row, one behind each other and...",6/25/2020
1,4/30/2020 23:49,Saint Petersburg,FL,Other,7 minutes,There were three dots in a straight line slowl...,6/25/2020
2,4/30/2020 22:43,Edgewood,WA,Unknown,15-20 seconds,"Location of sighting: Edgewood, WA Date: Thurs...",6/25/2020
3,4/30/2020 22:30,Vergas,MN,Light,>2 hours,White moving light. ((NUFORC Note: Venus?? PD)),6/25/2020
4,4/30/2020 22:00,Jacksonville,FL,Light,5 minutes,"Looked like stars, could have been satellites,...",6/25/2020


## Load NUFORC CSVs (1906-2014)

In [11]:
## Load National Ufo Reporting Center Data for 1906-1954
#nuforc_file_1 = "data/csv/nuforc_reports/nuforc_1906_54.csv"
#nuforc_1_df = pd.read_csv(nuforc_file_1)
#nuforc_1_df.head()

,datetime,city,state,country,shape,duration_seconds,duration_hours_min,comments,date_posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB TX. Lights racing across th...,12/16/2005,29.384210,-98.581082
2,10/1/1950 11:00,pikeville,ky,us,disk,600.0,hr,SEVENTY SOME FLYING DISK OBSERVED IN 1950,4/16/2005,37.479167,-82.518889
3,10/1/1952 3:30,fukuoka (japan),NaN,NaN,disk,1200.0,about 20 mins,UFO seen by multiple U. S. military personnel;...,12/7/2006,33.590355,130.401716
4,10/1/1952 12:00,kansas city,mo,us,disk,90.0,90 sec.,1952 daylight sighting of multiple discs in fo...,10/31/2008,39.099722,-94.578333


In [12]:
## Load National Ufo Reporting Center Data for 1955-1969
#nuforc_file_2 = "data/csv/nuforc_reports/nuforc_1955_69.csv"
#nuforc_2_df = pd.read_csv(nuforc_file_2)
#nuforc_2_df.head()

,datetime,city,state,country,shape,duration_seconds,duration_hours_min,comments,date_posted,latitude,longitude
0,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester England,1/21/2008,53.200000,-2.916667
1,10/10/1956 21:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833
2,10/10/1960 20:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611
3,10/10/1961 19:00,bristol,tn,us,sphere,300.0,5 minutes,My father is now 89 my brother 52 the girl wit...,4/27/2007,36.595000,-82.188889
4,10/10/1965 21:00,penarth (uk/wales),NaN,gb,circle,180.0,about 3 mins,penarth uk circle 3mins stayed 30ft above m...,2/14/2006,51.434722,-3.180000


In [13]:
## Load National Ufo Reporting Center Data for 1970-1984
#nuforc_file_3 = "data/csv/nuforc_reports/nuforc_1970_84.csv"
#nuforc_3_df = pd.read_csv(nuforc_file_3)
#nuforc_3_df.head()

,datetime,city,state,country,shape,duration_seconds,duration_hours_min,comments,date_posted,latitude,longitude
0,10/10/1970 16:00,bellmore,ny,us,disk,1800.0,30 min.,silver disc seen by family and neighbors,5/11/2000,40.668611,-73.527500
1,10/10/1970 19:00,manchester,ky,us,unknown,180.0,3 minutes,Slow moving silent craft accelerated at an unb...,2/14/2008,37.153611,-83.761944
2,10/10/1971 21:00,lexington,nc,us,oval,30.0,30 seconds,green oval shaped light over my local churchpo...,2/14/2010,35.823889,-80.253611
3,10/10/1972 19:00,harlan county,ky,us,circle,1200.0,20minutes,On october 10 1972 myselfmy 5yrs.daughter2 nei...,9/15/2005,36.843056,-83.321944
4,10/10/1972 22:30,west bloomfield,mi,us,disk,120.0,2 minutes,The UFO was so close my battery in the car wen...,8/14/2007,42.537778,-83.233056


In [15]:
## Load National Ufo Reporting Center Data for 1985-1999
#nuforc_file_4 = "data/csv/nuforc_reports/nuforc_85_99.csv"
#nuforc_4_df = pd.read_csv(nuforc_file_4)
#nuforc_4_df.head()

,datetime,city,state,country,shape,duration_seconds,duration_hours_min,comments,date_posted,latitude,longitude
0,10/10/1985 20:25,leeds (uk/england),NaN,gb,triangle,600.0,10min,three light in the sky that led to a big black...,3/4/2003,53.800000,-1.583333
1,10/10/1986 20:00,holmes/pawling,ny,NaN,chevron,180.0,3 minutes,Football Field Sized Chevron with bright white...,10/8/2007,41.523427,-73.646795
2,10/10/1988 18:00,milwaukee,wi,us,triangle,600.0,10 minutes,A silent black triangular object with no light...,8/7/2007,43.038889,-87.906389
3,10/10/1988 21:00,new gloucester,me,us,unknown,120.0,few minutes,I&#39m still afraid to look at the sky at night.,1/10/2009,43.962778,-70.283056
4,10/10/1988 22:00,boulder,co,us,light,60.0,1 minute,Three Stars Begin to Move Randomly Stop Change...,7/16/2006,40.015000,-105.270000


In [16]:
## Load National Ufo Reporting Center Data for 2000-2007
#nuforc_file_5 = "data/csv/nuforc_reports/nuforc_2000_07.csv"
#nuforc_5_df = pd.read_csv(nuforc_file_5)
#nuforc_5_df.head()

,datetime,city,state,country,shape,duration_seconds,duration_hours_min,comments,date_posted,latitude,longitude
0,10/10/2008 2:00,london (canada),on,ca,other,120.0,2 min. approx,C shape with a T front over London ont on Oct ...,6/9/2009,42.983333,-81.250000
1,10/10/2008 2:00,north branch,mn,us,light,300.0,5 min,Lights at night shining into car and garage an...,10/31/2008,45.511389,-92.980000
2,10/10/2008 2:00,slingerlands,ny,us,circle,240.0,4 min,Huge moon size orange bright almost complete c...,10/31/2008,42.629167,-73.865000
3,10/10/2008 4:00,carlin,nv,us,disk,600.0,10 mins,Ufo sighting in Carlin NV at the pilot truck s...,7/19/2010,40.713889,-116.103056
4,10/10/2008 4:50,evansville,in,us,formation,600.0,10 minutes+,I saw a huge V-shaped object moving slowly and...,10/31/2008,37.974722,-87.555833


In [17]:
## Load National Ufo Reporting Center Data for 2008-2014
#nuforc_file_6 = "data/csv/nuforc_reports/nuforc_08_14.csv"
#nuforc_6_df = pd.read_csv(nuforc_file_6)
#nuforc_6_df.head()

,datetime,city,state,country,shape,duration_seconds,duration_hours_min,comments,date_posted,latitude,longitude
0,10/10/2008 2:00,london (canada),on,ca,other,120.0,2 min. approx,C shape with a T front over London ont on Oct ...,6/9/2009,42.983333,-81.250000
1,10/10/2008 2:00,north branch,mn,us,light,300.0,5 min,Lights at night shining into car and garage an...,10/31/2008,45.511389,-92.980000
2,10/10/2008 2:00,slingerlands,ny,us,circle,240.0,4 min,Huge moon size orange bright almost complete c...,10/31/2008,42.629167,-73.865000
3,10/10/2008 4:00,carlin,nv,us,disk,600.0,10 mins,Ufo sighting in Carlin NV at the pilot truck s...,7/19/2010,40.713889,-116.103056
4,10/10/2008 4:50,evansville,in,us,formation,600.0,10 minutes+,I saw a huge V-shaped object moving slowly and...,10/31/2008,37.974722,-87.555833


## Connect to SQL DB

In [18]:
connection_string = "postgres:Postgres@localhost:5433/ufo_sightings"
engine = create_engine(f'postgresql://{connection_string}')

In [19]:
conn = engine.connect()

In [20]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

In [21]:
engine.table_names()

['nuforc_reports', 'military_bases', 'national_ufo_reporting_ctr']

## Query Data for Each Table

In [23]:
# Query All Records in the Nurc Database
april_ufo_data = engine.execute("SELECT * FROM national_ufo_reporting_ctr")

#for record in april_ufo_data:
#    print(record)

In [24]:
# Query All Records in the Nuforc Database 
nuforc_data = engine.execute("SELECT * FROM nuforc_reports")

#for record in nuforc_data:
#    print(record)

In [25]:
# Query All Records in the Military Bases Database
military_bases_data = engine.execute("SELECT * FROM military_bases")

# for record in militry_bases_data:
#    print(record)

## Get Column Names/Types for Each Table

In [26]:
# Get a list of column names and types for nurc data
columns = inspector.get_columns('national_ufo_reporting_ctr')
#for c in columns:
#    print(c['name'], c["type"])
# columns

In [27]:
# Get a list of column names and types for military_bases data
columns = inspector.get_columns('military_bases')
#for c in columns:
#    print(c['name'], c["type"])
# columns

In [28]:
# Get a list of column names and types for nuforc data
columns = inspector.get_columns('nuforc_reports')
#for c in columns:
#    print(c['name'], c["type"])
# columns

In [29]:
# Create the tables associated with classes.
Base.metadata.create_all(engine)

In [30]:
from sqlalchemy.orm import Session
session = Session(bind=engine)

## Reflect Tables Into SQLAlchemy ORM

In [31]:
# Declare a Base
Base = automap_base()

In [32]:
# reflect an existing database into a new model
Base.prepare(engine, reflect=True)

# reflect the tables

# We can view all of the classes that automap found
Base.classes.keys()

['nuforc_reports', 'military_bases', 'national_ufo_reporting_ctr']

In [33]:
# Save references to each table
nuforc_reports = Base.classes.nuforc_reports
military_bases = Base.classes.military_bases
national_ufo_reporting_ctr = Base.classes.national_ufo_reporting_ctr

In [34]:
# Create our session (link) from Python to the DB
session = Session(engine)

## Explore April Sightings

In [ ]:
# Total Sightings in April
national_ufo_reporting_ctr